In [1]:
import uuid
import random
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from create_sap_table import create_table_leanx as ctl

import values, helpers
import master_data, text_data, sales_doc_data

In [2]:
# master tables
for method in (
    master_data.users,
    master_data.customers_and_vendors, 
    master_data.plants, 
    master_data.materials, 
    master_data.material_support,
    master_data.routes,
    master_data.company_codes
):
    table_dict = method()
    for k, v in table_dict.items():
        table_name = k.split('_')[0]
        all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
        df = pd.concat([all_cols, pd.DataFrame(v.values())])
        df.to_csv(f'data/OCPM/master/{table_name}.csv', index=False)

KeyboardInterrupt: 

In [3]:
# text tables
for method in (
    text_data.domain_fixed_values, 
    text_data.doc_types, 
    text_data.organization,
    text_data.distribution,
    text_data.sales_doc_item_categories,
    text_data.sales_doc_rejection_reasons,
    text_data.system_status,
    text_data.blocking_reasons,
    text_data.releases
):
    table_dict = method()
    for k, v in table_dict.items():
        table_name = k.split('_')[0]
        all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
        df = pd.concat([all_cols, pd.DataFrame(v.values())])
        df.to_csv(f'data/OCPM/text/{table_name}.csv', index=False)

In [4]:
MARC = pd.read_csv('data/OCPM/master/MARC.csv')
MARA = pd.read_csv('data/OCPM/master/MARA.csv')
MAKT = pd.read_csv('data/OCPM/master/MAKT.csv')
KNB1 = pd.read_csv('data/OCPM/master/KNB1.csv')
KNA1 = pd.read_csv('data/OCPM/master/KNA1.csv')

all_prices = {}
all_availabilities = {}
all_material_groups = {}

# get price by MATNR
for nr in MARA['MATNR']:
    name = MAKT[MAKT['MATNR'] == nr]['MAKTX'].values[0]
    for k, v in values.om_material_groups.items():
        for mgrp, attr in v.items():
            for mat, details in attr['materials'].items():
                if name == mat:
                    all_prices[nr] = details['price']
                    all_availabilities[nr] = attr['availability']

def get_user_name(automation_probability: float):
    if random.random() <= automation_probability:
        return 'BATCH_JOB'
    return random.choice(list(values.om_users.keys()))

def get_time_consumption(start_date, planned_target_date, latest_date):
    return (latest_date - start_date) / (planned_target_date - start_date)

def get_params():
    company_code = random.choice(list(values.om_company_codes.keys()))
    plant = random.choice(values.om_company_codes[company_code]['plants'])
    all_matnrs = MARC[MARC['WERKS'] == plant]['MATNR'].unique()
    matnrs = random.sample(list(all_matnrs), min(random.randint(5, 25), len(all_matnrs)))
    quantities = [random.randint(12, 60)*12 for _ in range(len(matnrs))] # x dozens of everything
    prices = []
    availabilities = []
    
    for i in range(len(matnrs)):
        prices.append(all_prices[matnrs[i]])
        availabilities.append(all_availabilities[matnrs[i]])
    
    sales_org=random.choice(list(values.om_sales_orgs.keys()))
    sales_office=random.choice(list(values.om_sales_orgs[sales_org]['sales_offices'].keys()))
    distribution_channel=random.choice(list(values.om_sales_orgs[sales_org]['distribution_channels'].keys()))
    kunnr = random.choice(list(KNB1[KNB1['BUKRS'] == company_code]['KUNNR']))
    customer_name = KNA1[KNA1['KUNNR'] == kunnr]['NAME1'].values[0]
    payment_term = random.choice(list(KNB1[KNB1['KUNNR'] == kunnr]['ZTERM']))
    credit_risk = values.om_customers[customer_name]['credit_risk']
    sales_doc_type=random.choice(list(values.om_sales_doc_types.keys()))

    total = np.sum([prices[i]*quantities[i] for i in range(len(prices))])

    params = {
        'kunnr': kunnr,
        'credit_risk': credit_risk,
        'payment_term': payment_term,
        'company_code': company_code,
        'plant': plant,
        'matnrs': matnrs,
        'quantities': quantities,
        'prices': prices,
        'sales_org': sales_org,
        'sales_office': sales_office,
        'distribution_channel': distribution_channel,
        'availabilities': availabilities,
        'sales_doc_type': sales_doc_type,
        'total': total
    }

    return params

In [5]:
# sales order tables
sales_doc_tables = {
    'VBAK_json': {},
    'VBKD_json': {},
    'VBUK_json': {},
    'VBAP_json': {},
    'VBEP_json': {},
    'LIKP_json': {},
    'LIPS_json': {},
    'EKBE_json': {}, 
    'MSEG_json': {},
    'VBRK_json': {},
    'VBRP_json': {},
    'BKPF_json': {},
    'BSEG_json': {},
    'CDHDR_json': {},
    'CDPOS_json': {},
    'JCDS_json': {},
    'VBFA_json': {}
}

for i in range(1_00):
    transition_prob = np.array([
        [0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.05], # Approve Sales Order
        [0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.01], # Generate Delivery Document
        [0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.05], # Pick Items
        [0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.01], # Post Goods Issue
        [0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.05], # Send Invoice
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.01], # Recieve Delivery Confirmation
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 1.00], # Clear Invoice
        
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 1.00], # Reject Sales Order
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.75, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.25], # Set Credit Block
        [0.00, 0.90, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.10], # Release Credit Block
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.65, 0.00, 0.00, 0.00, 0.00,0.00, 0.35], # Set Delivery Block
        [0.00, 0.00, 0.90, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.10], # Release Delivery Block
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.85, 0.00, 0.00,0.00, 0.15], # Set Sales Order Billing Block
        [0.00, 0.00, 0.00, 0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.05], # Release Sales Order Billing Block
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.85,0.00, 0.15], # Set Customer Billing Block
        [0.00, 0.00, 0.00, 0.00, 0.90, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.10], # Release Customer Billing Block
        [0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.01], # Change Payment Term
        
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 1.00] # Terminate

    ])

    """
    Steps map
    Starts at:  Create Sales Order
    0: Approve Sales Order
    1: Generate Delivery Document
    2: Pick Items
    3: Post Goods Issue
    4: Send Invoice
    5: Recieve Delivery Confirmation
    6: Clear Invoice

    7: Reject Sales Order
    8: Set Credit Block
    9: Release Credit Block
    10: Set Delivery Block
    11: Release Delivery Block
    12: Set Sales Order Billing Block
    13: Release Sales Order Billing Block
    14: Set Customer Billing Block
    15: Release Customer Billing Block
    16: Change Payment Term

    17: Terminate Process
    """

    params=get_params()
    latest_date = helpers.generate_random_date(start_date=datetime(2021, 1, 1), end_date=datetime(2023, 9, 1))
    so_created_date = latest_date
    latest_time = helpers.generate_random_time()
    so_created_by = get_user_name(0.3)
    reqested_delivery_date=helpers.add_random_days(min_days=3, max_days=10, current_date=latest_date)

    sd = sales_doc_data.SalesAndDistribution(
        vbeln=f'{str(uuid.uuid4())[-17:]}', 
        params=params,
        start_date=latest_date
    )

    sd.create_sales_order(
        reqested_delivery_date=reqested_delivery_date,
        shipping_condition=random.choice(list(values.shipping_conditions.keys())),
        erdat=latest_date,
        ernam=so_created_by,
        atime=latest_time
    )
    # latest_date += helpers.UPTO_WEEK()
    latest_time = helpers.add_random_hours(1, latest_time)

    # decide on apprve or reject sales order
    def nex_step():
        if so_created_by != 'BATCH_JOB':
            if random.random() < 0.4:
                return 7 # Reject Sales Oreder for inaccurate information
        
        return 0 # Approve
    
    step = nex_step()
    while step != 17: # until termination
        if step == 0:
            sd.approve_sales_order(
                usnam=get_user_name(0.7),
                udate=latest_date,
                atime=latest_time
            )
            
            # Set Credit Block
            if params['credit_risk'] > 0.7:
                transition_prob[step][8] = 0.5
                latest_date = helpers.add_random_days(1, 3, latest_date)
            # Customer Billing Block
            elif params['credit_risk'] > 0.5:
                transition_prob[step][14] = 0.5
                latest_date = helpers.add_random_days(1, 3, latest_date)
            # Change Payment Term
            elif random.random() > 0.95:
                transition_prob[step][16] = 0.5
                latest_date = helpers.add_random_days(1, 3, latest_date)
            else:
                latest_date = helpers.add_random_days(1, 1, latest_date)
        elif step == 1:
            sd.generate_delivery_document(
                ernam=get_user_name(0.8), 
                erdat=latest_date,
                planned_delivery_date=latest_date + helpers.UPTO_WEEK(),
                picking_date=None,
                delivery_date=None,
                confirmation_date=None,
                atime=latest_time
            )
            latest_time = helpers.add_random_hours(3, latest_time)

            # Delivery blocks
            if np.average(params['availabilities']) < 0.7:
                latest_time = helpers.add_random_hours(1, latest_time)
                transition_prob[step][10] = 0.5
            else:
                latest_time = helpers.add_random_hours(1, latest_time)
        elif step == 2:
            sd.pick_items(
                usnam=get_user_name(0.2), 
                udate=latest_date,
                atime=helpers.generate_random_time()
            )
            latest_time = helpers.add_random_hours(8, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 3:
            sd.post_goods_issue(
                cpudt=latest_date,
                usnam=get_user_name(0.3),
                atime=helpers.generate_random_time()
            )

            # Sales Order Billing Block
            if params['total'] > 35_000:
                transition_prob[step][12] = 0.5
                latest_time = helpers.add_random_hours(12, latest_time)
                latest_date = helpers.add_random_days(0, reqested_delivery_date.day, latest_date)   
            else:
                latest_time = helpers.add_random_hours(2, latest_time)
                latest_date = helpers.add_random_days(0, reqested_delivery_date.day, latest_date)
        elif step == 4:
            sd.create_invoice(
                ernam=get_user_name(0.8),
                erdat=latest_date,
                atime=helpers.generate_random_time()
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 7, latest_date)
        elif step == 5:
            sd.delivery_confirmation(
                usnam=get_user_name(0.8), 
                udate=latest_date
            )
            latest_time = helpers.add_random_hours(1, latest_time)
        
            if get_time_consumption(start_date=so_created_date, planned_target_date=reqested_delivery_date, latest_date=latest_date) > 1.25:
                latest_date = helpers.add_random_days(0, 14, latest_date)
            else:
                latest_date = helpers.add_random_days(0, 7, latest_date)
        elif step == 6:
            sd.clear_debit_invoice(
                cpudt=latest_date,
                usnam=get_user_name(0.1),
                cleared_date=latest_date + helpers.UPTO_WEEK(),
                atime=helpers.generate_random_time()
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        
        elif step == 7:
            sd.reject_sales_order(
                udate=latest_date,
                usnam=get_user_name(0.9), 
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 8:
            sd.set_credit_block(
                udate=latest_date,
                usnam=get_user_name(0.2)
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 15, latest_date)
        elif step == 9:
            sd.release_credit_block(
                udate=latest_date,
                usnam=get_user_name(0.1), 
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 10:
            sd.set_delivery_block(
                udate=latest_date,
                usnam=get_user_name(0.7), 
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 30, latest_date)
        elif step == 11:
            sd.release_delivery_block(
                udate=latest_date,
                usnam=get_user_name(0.1), 
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 12:
            sd.set_sales_order_billing_block(
                udate=latest_date,
                usnam=get_user_name(0.9), 
                blocked_matnrs=[params['matnrs'][0]]
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 7, latest_date)
        elif step == 13:
            sd.release_sales_order_billing_block(
                udate=latest_date,
                usnam=get_user_name(0.1), 
                blocked_matnrs=[params['matnrs'][0]]
            )
            latest_time = helpers.add_random_hours(4, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 14:
            sd.set_customer_billing_block(
                udate=latest_date,
                usnam=get_user_name(0.7), 
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 14, latest_date)
        elif step == 15:
            sd.release_customer_billing_block(
                udate=latest_date,
                usnam=get_user_name(0.1), 
            )
            latest_time = helpers.add_random_hours(5, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 16:
            sd.change_payment_term(
                udate=latest_date,
                usnam=get_user_name(0.1), 
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 7, latest_date)

        # update transition matrix to sum to 1
        transition_prob = transition_prob / transition_prob.sum(axis=1, keepdims=True)
        step = np.random.choice(18, p=transition_prob[step])

    for k, v in sd.tables.items():
        for entry_key in list(v.keys()):
            sales_doc_tables[k][entry_key] = sd.tables[k][entry_key]

In [6]:
for table, rows in sales_doc_tables.items():
    table_name = table.split('_')[0]
    all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
    df = pd.concat([all_cols, pd.DataFrame(rows.values())])
    df.to_csv(f'data/OCPM/sales-document/{table_name}.csv', index=False)